In [2]:
%load_ext nb_black

<IPython.core.display.Javascript object>

## Day 34 Lecture 2 Assignment

In this assignment, we will learn about gradient boosting. We will use a dataset describing TripAdvisor reviews for Las Vegas hotels loaded below and analyze the model generated for this dataset.

In [1]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.ensemble import GradientBoostingRegressor

import ssl

# ssl._create_default_https_context = ssl._create_unverified_context

In [2]:
vegas = pd.read_csv(
    "https://tf-assets-prod.s3.amazonaws.com/tf-curric/data-science/LasVegasTripAdvisorReviews-Dataset.csv",
    sep=";",
)

URLError: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1056)>

In [94]:
vegas.head()

,User country,Nr. reviews,Nr. hotel reviews,Helpful votes,Score,Period of stay,Traveler type,Pool,Gym,Tennis court,Spa,Casino,Free internet,Hotel name,Hotel stars,Nr. rooms,User continent,Member years,Review month,Review weekday
0,USA,11,4,13,5,Dec-Feb,Friends,NO,YES,NO,NO,YES,YES,Circus Circus Hotel & Casino Las Vegas,3,3773,North America,9,January,Thursday
1,USA,119,21,75,3,Dec-Feb,Business,NO,YES,NO,NO,YES,YES,Circus Circus Hotel & Casino Las Vegas,3,3773,North America,3,January,Friday
2,USA,36,9,25,5,Mar-May,Families,NO,YES,NO,NO,YES,YES,Circus Circus Hotel & Casino Las Vegas,3,3773,North America,2,February,Saturday
3,UK,14,7,14,4,Mar-May,Friends,NO,YES,NO,NO,YES,YES,Circus Circus Hotel & Casino Las Vegas,3,3773,Europe,6,February,Friday
4,Canada,5,5,2,4,Mar-May,Solo,NO,YES,NO,NO,YES,YES,Circus Circus Hotel & Casino Las Vegas,3,3773,North America,7,March,Tuesday


<IPython.core.display.Javascript object>

Check for missing data and remove all rows with missing data

In [95]:
vegas.isna().mean()

User country         0.0
Nr. reviews          0.0
Nr. hotel reviews    0.0
Helpful votes        0.0
Score                0.0
Period of stay       0.0
Traveler type        0.0
Pool                 0.0
Gym                  0.0
Tennis court         0.0
Spa                  0.0
Casino               0.0
Free internet        0.0
Hotel name           0.0
Hotel stars          0.0
Nr. rooms            0.0
User continent       0.0
Member years         0.0
Review month         0.0
Review weekday       0.0
dtype: float64

<IPython.core.display.Javascript object>

How many reviews do we have for each hotel in the dataset?

In [96]:
vegas["Hotel name"].value_counts()

Caesars Palace                                         24
The Venetian Las Vegas Hotel                           24
Circus Circus Hotel & Casino Las Vegas                 24
Monte Carlo Resort&Casino                              24
Paris Las Vegas                                        24
The Westin las Vegas Hotel Casino & Spa                24
The Cosmopolitan Las Vegas                             24
The Palazzo Resort Hotel Casino                        24
Treasure Island- TI Hotel & Casino                     24
Excalibur Hotel & Casino                               24
Hilton Grand Vacations on the Boulevard                24
Trump International Hotel Las Vegas                    24
Bellagio Las Vegas                                     24
Wyndham Grand Desert                                   24
Tuscany Las Vegas Suites & Casino                      24
Marriott's Grand Chateau                               24
Tropicana Las Vegas - A Double Tree by Hilton Hotel    24
Wynn Las Vegas

<IPython.core.display.Javascript object>

We would like to predict the score variable. Examine the dataset and decide which columns should be turned into dummy variables and transform the data. Also, where we have two columns with redundant information, remove one of the two columns. Remove the hotel stars column.

In [97]:
vegas.columns

Index(['User country', 'Nr. reviews', 'Nr. hotel reviews', 'Helpful votes',
       'Score', 'Period of stay', 'Traveler type', 'Pool', 'Gym',
       'Tennis court', 'Spa', 'Casino', 'Free internet', 'Hotel name',
       'Hotel stars', 'Nr. rooms', 'User continent', 'Member years',
       'Review month', 'Review weekday'],
      dtype='object')

<IPython.core.display.Javascript object>

In [98]:
vegas.head(3)

,User country,Nr. reviews,Nr. hotel reviews,Helpful votes,Score,Period of stay,Traveler type,Pool,Gym,Tennis court,Spa,Casino,Free internet,Hotel name,Hotel stars,Nr. rooms,User continent,Member years,Review month,Review weekday
0,USA,11,4,13,5,Dec-Feb,Friends,NO,YES,NO,NO,YES,YES,Circus Circus Hotel & Casino Las Vegas,3,3773,North America,9,January,Thursday
1,USA,119,21,75,3,Dec-Feb,Business,NO,YES,NO,NO,YES,YES,Circus Circus Hotel & Casino Las Vegas,3,3773,North America,3,January,Friday
2,USA,36,9,25,5,Mar-May,Families,NO,YES,NO,NO,YES,YES,Circus Circus Hotel & Casino Las Vegas,3,3773,North America,2,February,Saturday


<IPython.core.display.Javascript object>

In [99]:
vegas = vegas.drop(columns=["Hotel stars", "User country"])
# vegas = vegas.drop(columns=["Hotel name"])

<IPython.core.display.Javascript object>

In [100]:
cols_to_encode = [
    "Review month",
    "Review weekday",
    "User continent",
    "Traveler type",
    "Period of stay",
    "Hotel name",
]

binary_cols = ["Pool", "Gym", "Tennis court", "Spa", "Casino", "Free internet"]

numeric_cols = [
    "Nr. reviews",
    "Nr. hotel reviews",
    "Helpful votes",
    "Nr. rooms",
    "Member years",
]

<IPython.core.display.Javascript object>

In [101]:
vegas = vegas.replace("YES", 1)
vegas = vegas.replace("NO", 0)

<IPython.core.display.Javascript object>

In [102]:
def one_hot_encode(X, encode_cols, numeric_cols, index=None):
    other_cols = [c for c in X.columns if c not in encode_cols]

    ct = ColumnTransformer(
        #   Format
        #   [("name of step", what_to_do(), [what columns to do it to])]
        [
            ("one hot encode", OneHotEncoder(drop="first", sparse=False), encode_cols),
            ("scaler", StandardScaler(), numeric_cols),
        ],
        remainder="passthrough",
    )

    ct.fit(X)
    print(ct)
    encoded_names = ct.transformers_[0][1].get_feature_names()
    encoded_names = list(encoded_names)

    X_encoded = ct.transform(X)
    X_encoded = pd.DataFrame(X_encoded, columns=encoded_names + other_cols, index=index)

    return X_encoded, ct

<IPython.core.display.Javascript object>

In [103]:
vegas, ct = one_hot_encode(vegas, cols_to_encode, numeric_cols, index=vegas.index)
ct

ColumnTransformer(n_jobs=None, remainder='passthrough', sparse_threshold=0.3,
                  transformer_weights=None,
                  transformers=[('one hot encode',
                                 OneHotEncoder(categories='auto', drop='first',
                                               dtype=<class 'numpy.float64'>,
                                               handle_unknown='error',
                                               sparse=False),
                                 ['Review month', 'Review weekday',
                                  'User continent', 'Traveler type',
                                  'Period of stay', 'Hotel name']),
                                ('scaler',
                                 StandardScaler(copy=True, with_mean=True,
                                                with_std=True),
                                 ['Nr. reviews', 'Nr. hotel reviews',
                                  'Helpful votes', 'Nr. rooms',
               

ColumnTransformer(n_jobs=None, remainder='passthrough', sparse_threshold=0.3,
                  transformer_weights=None,
                  transformers=[('one hot encode',
                                 OneHotEncoder(categories='auto', drop='first',
                                               dtype=<class 'numpy.float64'>,
                                               handle_unknown='error',
                                               sparse=False),
                                 ['Review month', 'Review weekday',
                                  'User continent', 'Traveler type',
                                  'Period of stay', 'Hotel name']),
                                ('scaler',
                                 StandardScaler(copy=True, with_mean=True,
                                                with_std=True),
                                 ['Nr. reviews', 'Nr. hotel reviews',
                                  'Helpful votes', 'Nr. rooms',
               

<IPython.core.display.Javascript object>

In [ ]:
ct

Split the data into train and test (20% in test)

In [75]:
X = vegas.drop(columns=["Score"])
y = vegas["Score"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=33
)

<IPython.core.display.Javascript object>

Create a graident boosted regression model for predicting the score. To produce the accuracy score for the test data, first round the prediction and then compare to the observed.

In [76]:
model = GradientBoostingRegressor()
model.fit(X_train, y_train)

print(model.score(X_train, y_train), "\n")
print(model.score(X_test, y_test))

0.9782654573401339 

0.9501786173759806


<IPython.core.display.Javascript object>

In [77]:
y_pred= model.predict(X_test)

errors= y_pred- y_test
mae= np.abs(errors).mean()
mae


0.12572072365779688

<IPython.core.display.Javascript object>

Try again with a learning rate of 0.8 and 0.3 and compare the results.

In [78]:
model = GradientBoostingRegressor(learning_rate=0.8)
model.fit(X_train, y_train)

print(model.score(X_train, y_train), "\n")
print(model.score(X_test, y_test))

0.9998783599387276 

0.9616369118013208


<IPython.core.display.Javascript object>